# State Farm Distracted Driver Detection
This notebook contains code for State Farm Distracted Driver Detection dataset chanllenge. Kaggle link: https://www.kaggle.com/c/state-farm-distracted-driver-detection

## Importing packages
Install and import necessary libraries

In [1]:
!pip install --upgrade pip
!pip install tqdm matplotlib pandas

DEPRECATION: Python 3.5 reached the end of its life on September 13th, 2020. Please upgrade your Python as Python 3.5 is no longer maintained. pip 21.0 will drop support for Python 3.5 in January 2021. pip 21.0 will remove support for this functionality.
  Using cached pip-20.3.4-py2.py3-none-any.whl (1.5 MB)
  Using cached pip-20.3.3-py2.py3-none-any.whl (1.5 MB)
DEPRECATION: Python 3.5 reached the end of its life on September 13th, 2020. Please upgrade your Python as Python 3.5 is no longer maintained. pip 21.0 will drop support for Python 3.5 in January 2021. pip 21.0 will remove support for this functionality.


In [1]:
import os
import shutil
import glob

import tqdm
import pandas as pd
import cv2
import caffe
import lmdb
import numpy as np
from sklearn.model_selection import train_test_split
from caffe.proto import caffe_pb2
from matplotlib import pyplot as plt

%matplotlib inline

## Preprocessing data
For large dataset we usually split the dataset into 3 subsets: training, validation and testing. We already have a individual test set for final evaluation. So we still need to split the original training set into a training and a validation set for parameters tuning.

As you see the datas are already seperated into each class: one directory for images with label c0, another directory for images with label c1, etc. For loading the dataset thus we don't need the .csv file provided, we can go to the directories one by one and load the images, and for each image we record the parent directory name (c0, c1, ...) as the label for that image.

In [ ]:
def prepare_data(data_dir, split=0.2):
    """Load raw data and split it into training and validation subset.
    
    Args
    :data_dir: Data root directory.
    
    Returns
    :X_train: A list of training image paths.
    :y_train: A list of training labels.
    :X_val: A list of validation image paths.
    :y_val: A list of validation labels.
    """
    imgs_list = []
    labels = []

    # List all image subdirectories and sort by class name
    img_dirs = sorted(glob.glob(os.path.join(data_dir, '*')), key = lambda k: k.split("/")[-1])
    for img_dir in img_dirs:
        # Read all the images in this class
        # Image subdirectory name as label
        for img_path in glob.glob(os.path.join(img_dir,'*.jpg')):
            imgs_list.append(img_path)
            labels.append(int(img_dir.split("/")[-1].replace('c', '')))
    
    # Split into training and validation subset
    X_train, X_test, y_train, y_test = train_test_split(imgs_list, labels, test_size=0.2, random_state=32)

    return np.array(X_train), np.array(X_test), y_train, y_test

## Get data
Here we use the function we just defined above to load data. Make sure you placed the dataset in docker container.

In [ ]:
path_train_images = 'imgs/train'
path_test_images = 'imgs/test'

X_train, X_test, y_train, y_test = prepare_data(path_train_images)

print('Size of X_train: {}, size of y_train: {}'.format(len(X_train), len(y_train)))
print('Size of X_test: {}, size of y_test: {}'.format(len(X_test), len(y_test)))

## Data sanity check

Classes:
- c0: safe driving
- c1: texting - right
- c2: talking on the phone - right
- c3: texting - left
- c4: talking on the phone - left
- c5: operating the radio
- c6: drinking
- c7: reaching behind
- c8: hair and makeup
- c9: talking to passenger

In [ ]:
# Load an image
img = cv2.imread(X_train[0])
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.axis('off')
plt.show()

# Check label
print('Class: {}'.format(y_train[0]))

## Data Exploring
It's common in Machine Learning to see how balanced the dataset is. It's the best if our dataset is perfectly balanced. Otherwise we need to apply techniques to address it.

In [ ]:
path_label_csv = 'driver_imgs_list.csv'
data_file = pd.read_csv(path_label_csv)
data_x = list(pd.unique(data_file['classname']))

# Clustring all images of each class together
data_classes = data_file.loc[:, ['classname','img']].groupby(by='classname').count().reset_index()
data_y =list(data_classes['img'])

# Plotting them using matplot
plt.rcParams.update({'font.size': 22})
plt.figure(figsize=(30,10))
plt.bar(data_x, data_y)
plt.ylabel('Count classes')
plt.title('Classes')
plt.xticks(rotation=45)

Lucky us! The dataset is very nice. All classes are nearly equal in quantity.

## Caffe Overview
Caffe is a deep learning framework developed by the Berkeley Vision and Learning Center (BVLC). It is written in C++ and has Python and Matlab interfaces.

There are 4 steps in training a CNN using Caffe:

- Step 1 - Data preparation: In this step, we get the images and store them in a format that can be used by Caffe. Here we will write a Python script that will handle image storage.

- Step 2 - Model definition: In this step, we choose a CNN architecture and we define its parameters in a configuration file with extension .prototxt.

- Step 3 - Solver definition: The solver is responsible for model optimization. We define the solver parameters in a configuration file with extension .prototxt.

- Step 4 - Model training: We train the model by executing caffe command from the terminal. After training the model, we will get the trained model in a file with extension .caffemodel.

After the training phase, we will use the .caffemodel trained model to make predictions of new unseen data.

## Data preparation
Here we prepare the raw dataset as LMDB database, which is standard Caffe data format. We need some piece of Python code.

In [ ]:
import lmdb
import caffe
from caffe.proto import caffe_pb2


def transform_img(img, img_width=128, img_height=128):
    """Resize and normalize image.
    
    Args
    :img: numpy array image.
    :img_width: Target image width.
    :img_height: Target image height.
    
    Returns
      transformed image.
    """
    img = cv2.resize(img, (img_width, img_height), interpolation=cv2.INTER_CUBIC)

    # TODO: normalize
    return img


def make_datum(img, label):
    """
    Convert original numpy array image to datum
    Args
    :img: numpy.ndarray (BGR instead of RGB)
    :label: int
    """
    return caffe_pb2.Datum(
        channels=3,
        width=128,
        height=128,
        label=label,
        data=np.rollaxis(img, 2).tostring())


def make_lmdb(lmdb_path, x_data, y_data):
    """Create LMDB database from the given raw images and labels.
    
    Args
    :lmdb_path: LMDB output path.
    :x_data: A list of image paths.
    :y_data: A list of labels.
    """
    in_db = lmdb.open(lmdb_path, map_size=int(1e12))
    with in_db.begin(write=True) as in_txn:
        for in_idx, img_path in tqdm.tqdm(enumerate(x_data)):
            img = cv2.imread(img_path)
            img = transform_img(img)
            datum = make_datum(img, y_data[in_idx])  # Making datum object
            in_txn.put('{:0>5d}'.format(in_idx).encode('utf-8'), datum.SerializeToString())
    in_db.close()

In [ ]:
# Actually create training and validation database
train_lmdb = 'input/train_lmdb'
val_lmdb = 'input/validation_lmdb'

os.makedirs(train_lmdb, exist_ok=True)
os.makedirs(val_lmdb, exist_ok=True)
make_lmdb(train_lmdb, X_train, y_train)
make_lmdb(val_lmdb, X_test, y_test)

## Create architecture
Caffe philosophy is expressivity and speed. For that we use text files to define networks, instead of code API like Keras. Coding is possible in Caffe too, but highly discoureged.

After deciding on the CNN architecture, we need to define its parameters in a .prototxt file. Here is the details of the defined network structure in my git repo.

### 1. Data Layer
Data enters Caffe through data layers: they lie at the bottom of nets. Data can come from efficient databases (LevelDB or LMDB), directly from memory, or, when efficiency is not critical, from files on disk in HDF5 or common image formats. Parameters we have in data layer:

- source: the path to the datas it needs to read
- backend: specifies the data type that we read
- batch_size: specifies the size of image batches to read at each step
- transform_param: input transformation params. In computer vision task, it's common to normalize input images. Here we provide 3 mean values correspond to 3 image channels. `mirror` means horizontal flip augmentation.
```
layer {
  name: “data”
  type: “Data”
  include {
    phase: TRAIN   # Or TEST
  }
  data_param {
    source: "./input/train_lmdb"
    backend: LMDB
    batch_size: 32
  }
  top: “data”
  top: “label”
  transform_param {
    crop_size: 224
    mean_value: 104
    mean_value: 117
    mean_value: 123
    mirror: true
 }
}
```
### 2. Convolution layer
This layer recieves the data blob from last layer and produces conv1 blob. Convolution layers in neural networks generally convolve the input image with a set of learnable filters, each producing one feature map in the output image.

This layer produces 64 filters and kernel size is 3 with the stride of 1 done on input. Fillers help us initialize weight and bias values randomly. Here we use Xavier algorithm to automatically initialize weights based on the number of input and output neurons. And for bias we use a simple constant number of zero. lr_mult is also the settings for learning rate, here we set the learning rate for weights same as the resolver in runtime and the learning rate for bias twice of that.

```
layer {
  name: "conv1"
  type: "Convolution"
  param { lr_mult: 1 }
  param { lr_mult: 2 }
  convolution_param {
    num_output: 64
    kernel_size: 3
    stride: 1
    weight_filler {
      type: "xavier"
    }
    bias_filler {
      type: "constant"
    }
  }
  bottom: "data"
  top: "conv1"
}
```
### 3. Pooling layer
We set the pool to max so it does max pooling operation on convolution outputs.

```
layer {
  name: "pool1"
  type: "Pooling"
  pooling_param {
    kernel_size: 2
    stride: 2
    pool: MAX
  }
  bottom: "conv1"
  top: "pool1"
}
```
### 4. Dense layer
This layer is similar to previous layers too. Dense layers are knows as InnerProduct layers in Caffe. Here we have a dense layer which has 128 output and parameters is same as previous layers explained.

```
layer {
  name: "ip1"
  type: "InnerProduct"
  param { lr_mult: 1 }
  param { lr_mult: 2 }
  inner_product_param {
    num_output: 128
    weight_filler {
      type: "xavier"
    }
    bias_filler {
      type: "constant"
    }
  }
  bottom: "pool2"
  top: "ip1"
}
```
### 5. ReLU layer
Since ReLU is element-wise we can do the operation once and not waste memory. This can be done with defining one name for top and bottom layers. Note that we can not have same names for blob of other layers and this is pecuilar for this layer.
```
layer {
  name: "relu1"
  type: "ReLU"
  bottom: "ip1"
  top: "ip1"
}
```

After ReLU we define another Dense layer with bottom: "ip1" and top: "ip2"

### 6. Loss
We define loss as follow:
```
layer {
  name: "loss"
  type: "SoftmaxWithLoss"
  bottom: "ip2"
  bottom: "label"
}
```

## Caffe Solver
The solver is responsible for model optimization. We define the solver's parameters in a .prototxt file. 

This solver computes the accuracy of the model using the validation set every 1000 iterations. The optimization process will run for a maximum of 40000 iterations and will take a snapshot of the trained model every 5000 iterations.

base_lr, lr_policy, gamma, momentum and weight_decay are hyperparameters that we need to tune to get a good convergence of the model.

we chose lr_policy: "step" with stepsize: 2500, base_lr: 0.001 and gamma: 0.1. In this configuration, we will start with a learning rate of 0.001, and we will drop the learning rate by a factor of ten every 2500 iterations.

There are different strategies for the optimization process. For a detailed explanation, you can read Caffe's solver documentation.
```
net: "caffe-cnn/cnn/cnn.prototxt"
test_interval: 1000
base_lr: 0.001
lr_policy: "step"
gamma: 0.1
stepsize: 2500
display: 50
max_iter: 40000
momentum: 0.9
weight_decay: 0.0005
snapshot: 5000
snapshot_prefix: "./snapshot/cnn"
solver_mode: CPU

```

## Training
We defined the network archirecture and solver in git repo. So just clone it and run the solver.

In [3]:
!git clone https://github.com/tamnguyenvan/caffe-cnn

Cloning into 'caffe-cnn'...
remote: Enumerating objects: 201, done.
remote: Counting objects: 100% (201/201), done.
remote: Compressing objects: 100% (195/195), done.
remote: Total 201 (delta 94), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (201/201), 213.37 KiB | 263.00 KiB/s, done.
Resolving deltas: 100% (94/94), done.
Checking connectivity... done.


In [4]:
!mkdir -p ./snapshot/cnn
!caffe train --solver "caffe-cnn/cnn/cnn_solver.prototxt"

I0124 08:17:32.396397   108 upgrade_proto.cpp:1113] snapshot_prefix was a directory and is replaced to ./snapshot/cnn/cnn_solver
I0124 08:17:32.396842   108 caffe.cpp:197] Use CPU.
I0124 08:17:32.397048   108 solver.cpp:45] Initializing solver from parameters: 
test_iter: 141
test_interval: 1000
base_lr: 0.001
display: 50
max_iter: 40000
lr_policy: "step"
gamma: 0.1
momentum: 0.9
weight_decay: 0.0005
stepsize: 2500
snapshot: 5000
snapshot_prefix: "./snapshot/cnn/cnn_solver"
solver_mode: CPU
net: "caffe-cnn/cnn/cnn.prototxt"
train_state {
  level: 0
  stage: ""
}
I0124 08:17:32.397346   108 solver.cpp:102] Creating training net from net file: caffe-cnn/cnn/cnn.prototxt
I0124 08:17:32.397657   108 net.cpp:294] The NetState phase (0) differed from the phase (1) specified by a rule in layer data
I0124 08:17:32.397697   108 net.cpp:294] The NetState phase (0) differed from the phase (1) specified by a rule in layer accuracy
I0124 08:17:32.397855   108 net.cpp:51] Initializing net from param

I0124 08:17:32.513347   108 net.cpp:122] Setting up fc5
I0124 08:17:32.513515   108 net.cpp:129] Top shape: 32 512 (16384)
I0124 08:17:32.513536   108 net.cpp:137] Memory required for data: 280035456
I0124 08:17:32.513566   108 layer_factory.hpp:77] Creating layer relu5
I0124 08:17:32.513604   108 net.cpp:84] Creating Layer relu5
I0124 08:17:32.513622   108 net.cpp:406] relu5 <- fc5
I0124 08:17:32.513762   108 net.cpp:367] relu5 -> fc5 (in-place)
I0124 08:17:32.513790   108 net.cpp:122] Setting up relu5
I0124 08:17:32.513808   108 net.cpp:129] Top shape: 32 512 (16384)
I0124 08:17:32.513821   108 net.cpp:137] Memory required for data: 280100992
I0124 08:17:32.513836   108 layer_factory.hpp:77] Creating layer drop5
I0124 08:17:32.513855   108 net.cpp:84] Creating Layer drop5
I0124 08:17:32.513870   108 net.cpp:406] drop5 <- fc5
I0124 08:17:32.513963   108 net.cpp:367] drop5 -> fc5 (in-place)
I0124 08:17:32.514000   108 net.cpp:122] Setting up drop5
I0124 08:17:32.514019   108 net.cpp:12

I0124 08:17:32.579360   108 net.cpp:122] Setting up fc5
I0124 08:17:32.579392   108 net.cpp:129] Top shape: 32 512 (16384)
I0124 08:17:32.579397   108 net.cpp:137] Memory required for data: 280035712
I0124 08:17:32.579411   108 layer_factory.hpp:77] Creating layer relu5
I0124 08:17:32.579421   108 net.cpp:84] Creating Layer relu5
I0124 08:17:32.579428   108 net.cpp:406] relu5 <- fc5
I0124 08:17:32.579437   108 net.cpp:367] relu5 -> fc5 (in-place)
I0124 08:17:32.579447   108 net.cpp:122] Setting up relu5
I0124 08:17:32.579452   108 net.cpp:129] Top shape: 32 512 (16384)
I0124 08:17:32.579455   108 net.cpp:137] Memory required for data: 280101248
I0124 08:17:32.579459   108 layer_factory.hpp:77] Creating layer drop5
I0124 08:17:32.579470   108 net.cpp:84] Creating Layer drop5
I0124 08:17:32.579475   108 net.cpp:406] drop5 <- fc5
I0124 08:17:32.579480   108 net.cpp:367] drop5 -> fc5 (in-place)
I0124 08:17:32.579488   108 net.cpp:122] Setting up drop5
I0124 08:17:32.579522   108 net.cpp:12

During the training process, we need to monitor the loss and the model accuracy. We can stop the process at anytime by pressing stop button. Caffe will take a snapshot of the trained model every 5000 iterations, and store them under `./snapshot/cnn` folder.

The snapshots have .caffemodel extension. For example, 5000 iterations snapshot will be called: `cnn_iter_5000.caffemodel`

## Prediction
We will use the trained model to make prediction on test data.

In [2]:
import caffe

net = caffe.Net('caffe-cnn/cnn/cnn.prototxt',
                './snapshot/cnn/cnn_solver_iter_13.caffemodel', caffe.TEST)

In [3]:
out = net.forward()
acc, loss = out['accuracy'], out['loss']
print('Accuracy: {:.2f} Loss: {:.4f}'.format(acc*100, loss))

Accuracy: 0.00 Loss: 2.3903


Test on a single image

In [6]:
def predict_on_single_image(net, img_path):
    """Return predicted class on a single image.
    
    Args
    :net: model instance.
    :img_path: Path to the image
    
    Returns
    :prob: The output probabilities of classes
    """
    input_shape = list(net.blobs['data'].shape)
    h, w = input_shape[-2:]
    img = caffe.io.load_image(img_path)
    img = cv2.resize(img, (w, h))
    img = np.transpose(img, (2, 0, 1))
    img = np.expand_dims(img, axis=0)
    net.blobs['data'].data[...] = img
    out = net.forward()
    pred = out['prob'][0]  # Single image
    return pred

In [7]:
import caffe

net = caffe.Net('caffe-cnn/cnn/cnn_deploy.prototxt',
                './snapshot/cnn/cnn_solver_iter_13.caffemodel', caffe.TEST)

img_path = './imgs/test/img_1.jpg'
pred = predict_on_single_image(net, img_path)
print('Class:', np.argmax(pred))

Class: 5


## Transfer Learning
Caffe comes with a repository that is used by researchers and machine learning practitioners to share their trained models. This library is called Model Zoo.

Using this command we download the CaffeNet network structure, trained on ImageNet dataset.

In [ ]:
# or download it manually
# !gdown https://drive.google.com/uc?id=1s1pBVYak923NPB7EP1TAaVQn-hj-S2Vx

In [8]:
!caffe train --solver="caffe-cnn/vgg-like/solver.prototxt" --weights "bvlc_reference_caffenet.caffemodel"

I0124 08:30:13.292640   332 caffe.cpp:197] Use CPU.
I0124 08:30:13.293282   332 solver.cpp:45] Initializing solver from parameters: 
test_iter: 141
test_interval: 1000
base_lr: 0.001
display: 50
max_iter: 40000
lr_policy: "step"
gamma: 0.1
momentum: 0.9
weight_decay: 0.0005
stepsize: 2500
snapshot: 5000
snapshot_prefix: "./snapshot/vgg-like"
solver_mode: CPU
net: "caffe-cnn/vgg-like/net.prototxt"
train_state {
  level: 0
  stage: ""
}
weights: "bvlc_reference_caffenet.caffemodel"
I0124 08:30:13.293772   332 solver.cpp:102] Creating training net from net file: caffe-cnn/vgg-like/net.prototxt
I0124 08:30:13.294454   332 net.cpp:294] The NetState phase (0) differed from the phase (1) specified by a rule in layer data
I0124 08:30:13.294539   332 net.cpp:294] The NetState phase (0) differed from the phase (1) specified by a rule in layer accuracy
I0124 08:30:13.294970   332 net.cpp:51] Initializing net from parameters: 
name: "CaffeNet"
state {
  phase: TRAIN
  level: 0
  stage: ""
}
layer 

I0124 08:30:13.347113   332 net.cpp:122] Setting up conv3
I0124 08:30:13.347169   332 net.cpp:129] Top shape: 32 384 7 7 (602112)
I0124 08:30:13.347185   332 net.cpp:137] Memory required for data: 54304896
I0124 08:30:13.347213   332 layer_factory.hpp:77] Creating layer relu3
I0124 08:30:13.347234   332 net.cpp:84] Creating Layer relu3
I0124 08:30:13.347252   332 net.cpp:406] relu3 <- conv3
I0124 08:30:13.347270   332 net.cpp:367] relu3 -> conv3 (in-place)
I0124 08:30:13.347290   332 net.cpp:122] Setting up relu3
I0124 08:30:13.347306   332 net.cpp:129] Top shape: 32 384 7 7 (602112)
I0124 08:30:13.347321   332 net.cpp:137] Memory required for data: 56713344
I0124 08:30:13.347337   332 layer_factory.hpp:77] Creating layer conv4
I0124 08:30:13.347358   332 net.cpp:84] Creating Layer conv4
I0124 08:30:13.347373   332 net.cpp:406] conv4 <- conv3
I0124 08:30:13.347390   332 net.cpp:380] conv4 -> conv4
I0124 08:30:13.361053   332 net.cpp:122] Setting up conv4
I0124 08:30:13.368619   332 net

I0124 08:30:13.987154   332 upgrade_proto.cpp:46] Attempting to upgrade input file specified using deprecated transformation parameters: bvlc_reference_caffenet.caffemodel
I0124 08:30:13.987182   332 upgrade_proto.cpp:49] Successfully upgraded file specified using deprecated data transformation parameters.
W0124 08:30:13.987190   332 upgrade_proto.cpp:51] Note that future Caffe releases will only support transform_param messages for transformation fields.
I0124 08:30:13.987419   332 upgrade_proto.cpp:55] Attempting to upgrade input file specified using deprecated V1LayerParameter: bvlc_reference_caffenet.caffemodel
I0124 08:30:14.264775   332 upgrade_proto.cpp:63] Successfully upgraded file specified using deprecated V1LayerParameter
I0124 08:30:14.277973   332 net.cpp:744] Ignoring source layer fc6
I0124 08:30:14.286594   332 net.cpp:744] Ignoring source layer fc8
I0124 08:30:14.317555   332 solver.cpp:190] Creating test net (#0) specified by net file: caffe-cnn/vgg-like/net.prototxt


I0124 08:30:14.433934   332 net.cpp:122] Setting up fc6-new
I0124 08:30:14.433965   332 net.cpp:129] Top shape: 32 4096 (131072)
I0124 08:30:14.433969   332 net.cpp:137] Memory required for data: 65560960
I0124 08:30:14.433977   332 layer_factory.hpp:77] Creating layer relu6
I0124 08:30:14.433986   332 net.cpp:84] Creating Layer relu6
I0124 08:30:14.433991   332 net.cpp:406] relu6 <- fc6-new
I0124 08:30:14.433997   332 net.cpp:367] relu6 -> fc6-new (in-place)
I0124 08:30:14.434005   332 net.cpp:122] Setting up relu6
I0124 08:30:14.434011   332 net.cpp:129] Top shape: 32 4096 (131072)
I0124 08:30:14.434015   332 net.cpp:137] Memory required for data: 66085248
I0124 08:30:14.434020   332 layer_factory.hpp:77] Creating layer drop6
I0124 08:30:14.434026   332 net.cpp:84] Creating Layer drop6
I0124 08:30:14.434029   332 net.cpp:406] drop6 <- fc6-new
I0124 08:30:14.434033   332 net.cpp:367] drop6 -> fc6-new (in-place)
I0124 08:30:14.434039   332 net.cpp:122] Setting up drop6
I0124 08:30:14.4

I0124 08:30:15.120971   332 solver.cpp:351] Iteration 0, Testing net (#0)
I0124 08:32:05.797765   341 data_layer.cpp:73] Restarting data prefetching from start.
I0124 08:32:09.314066   332 solver.cpp:418]     Test net output #0: accuracy = 0
I0124 08:32:09.314137   332 solver.cpp:418]     Test net output #1: loss = 2.30258 (* 1 = 2.30258 loss)
I0124 08:32:11.419153   332 solver.cpp:239] Iteration 0 (-1.4013e-45 iter/s, 116.349s/50 iters), loss = 2.30215
I0124 08:32:11.419318   332 solver.cpp:258]     Train net output #0: loss = 2.30215 (* 1 = 2.30215 loss)
I0124 08:32:11.419355   332 sgd_solver.cpp:112] Iteration 0, lr = 0.001
^C


Test this transfer learning model

In [9]:
import caffe

net = caffe.Net('caffe-cnn/vgg-like/net.prototxt',
                './snapshot/vgg-like_iter_36.caffemodel', caffe.TEST)

In [10]:
out = net.forward()
acc, loss = out['accuracy'], out['loss']
print('Accuracy: {:.2f} Loss: {:.2f}'.format(acc, loss))

Accuracy: 0.06 Loss: 2.30


Test on single image only

In [11]:
import caffe

net = caffe.Net('caffe-cnn/vgg-like/net_deploy.prototxt',
                './snapshot/vgg-like_iter_36.caffemodel', caffe.TEST)

img_path = './imgs/test/img_1.jpg'
pred = predict_on_single_image(net, img_path)

print('Class:', np.argmax(pred))

Class: 4
